# Darcy equation

In this tutorial we present how to solve a Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1}(\psi) {q} + \nabla \psi = -\nabla z\\
\partial_t \theta (\psi) + \nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
with boundary conditions:
$$ \psi = 0 \text{ on } \partial_{top} \Omega \qquad \psi = 1 \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

### Import and parameters

In [1]:
import shutil
import os

import numpy as np
import scipy.sparse as sps
import sympy as sp

import porepy as pp
import pygeon as pg

from math import ceil

In [2]:
K = 1000
N = 10
eps = 1e-6

output_directory = 'output_evolutionary_v1'

In [3]:
rho = 1000
g = pp.GRAVITY_ACCELERATION


T = 3/16
dt = 1/48

dt_D = 1/16

In [4]:
n = 2.06
alpha = 0.423
K_s = 4.96e-2

theta_r = 0.131
theta_s = 0.396

### $\theta$ and $k$, with their derivatives w.r.t. $\psi$

In [5]:
def theta(psi):
    th = np.ones(len(psi)) * theta_s
    th[psi <= 0] = theta_r + (theta_s - theta_r) * (1 / np.power((1 + np.power(-alpha * psi[psi <= 0], n)), 1-1/n) )

    return th

In [6]:
def dtheta_dpsi(psi):
    psi_var = sp.Symbol('psi', negative=True)
    diff = sp.diff( theta_r + (theta_s - theta_r) * 1 / (1+(-alpha * psi_var) ** n) ** (1-1/n), psi_var)
    
    res = np.zeros(shape=psi.shape)
    res[psi <= 0] = np.array(list(diff.subs(psi_var, val).evalf() for val in psi[psi <= 0]))

    return res

In [7]:
def d2theta_dpsi(psi):
    psi_var = sp.Symbol('psi', negative=True)
    diff = sp.diff( theta_r + (theta_s - theta_r) * 1 / (1+(-alpha * psi_var) ** n) ** (1-1/n), psi_var, 2)
    
    res = np.zeros(shape=psi.shape)
    res[psi <= 0] = np.array(list(diff.subs(psi_var, val).evalf() for val in psi[psi <= 0]))

    return res

In [8]:
def conductivity_tensor(psi):
    coeff = K_s * np.ones(len(psi))
    th = theta(psi[psi <= 0])

    coeff[psi <= 0] = np.array( K_s * np.power(th, 0.5) * np.power( 1 - np.power( 1 - np.power(th, n/(n-1)) , 1-1/n) , 2) )

    return pp.SecondOrderTensor(coeff)

In [9]:
def dconductivity_tensor_dpsi(psi):
    coeff = np.zeros(len(psi))
    psi_var = sp.Symbol('psi', negative=True)
    exp = theta_r + (theta_s - theta_r) * 1 / (1+(-alpha * psi_var) ** n) ** (1-1/n)

    diff = sp.diff( K_s * exp ** (1/2) * ( 1 - (1 - exp ** (n / (n-1))) ** (1-1/n) ) ** 2, psi_var)

    coeff[psi <= 0] = np.array(list(diff.subs(psi_var, val).evalf() for val in psi[psi <= 0]))

    return pp.SecondOrderTensor(coeff)

### Domain preparation

In [10]:
subdomain = pp.StructuredTriangleGrid([2*N, 3*N], [2,3])
# convert the grid into a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([subdomain])

In [11]:
key = "flow"

bc_value = []
bc_essential = []
initial_pressure = [ ]

gravity = []

RT0 = pg.RT0(key)
P0  = pg.PwConstants(key)

In [12]:
def g_func(x): 
    return np.array([0, -1, -1])

In [13]:
def initial_pressure_func(x): 
    return 1-x[1]

In [14]:
for subdomain, data in mdg.subdomains(return_data=True):
    g_proj = RT0.interpolate(subdomain, g_func)
    gravity.append(RT0.assemble_mass_matrix(subdomain) @ g_proj)
    initial_pressure.append(P0.interpolate(subdomain, initial_pressure_func))
        
    # with the following steps we identify the portions of the boundary
    # to impose the boundary conditions
    gamma_d1 = np.logical_and(subdomain.face_centers[0, :] >= 0, subdomain.face_centers[0, :] <= 1, subdomain.face_centers[1, :] == 3)
    gamma_d2 = np.logical_and(subdomain.face_centers[0, :] == 2, subdomain.face_centers[1, :] >= 0, subdomain.face_centers[1, :] <= 1)

    gamma_d = np.logical_or(gamma_d1, gamma_d2)
    gamma_n = np.logical_not(gamma_d)
    
    def psi_bc(x): return 1

    def bc_gamma_d(x, t):
        if   x[0] == 2 and x[1] >= 0 and x[1] <= 1:
            return 1 - x[1]
        elif x[1] == 3 and x[0] >= 0 and x[0] <= 1:
            if t < dt_D:
                return -2 + 2.2 * t / dt_D
            else:
                return 0.2
        else:
            return 0

    bc_value.append(lambda t: - RT0.assemble_nat_bc(subdomain, lambda x: bc_gamma_d(x,t), gamma_d))

    essential_pressure_dofs = np.zeros(P0.ndof(subdomain), dtype=bool)
    bc_essential.append(np.hstack((gamma_n, essential_pressure_dofs)))

A quick recap:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1}(\psi) {q} + \nabla \psi = -\nabla z\\
\partial_t \theta (\psi) + \nabla \cdot {q} = 0
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
becames:
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
N_0^{k} q_{k+1}^{n+1} - B^T \psi_{k+1}^{n+1} = G^{n+1} - N_1^{k} q_k^{n+1}\\
B q_{k+1}^{n+1} + \frac{M_1^k}{\Delta t} \psi_{k+1}^{n+1} = F^{n+1} - \frac{M_2^k}{\Delta t} \psi_{k}^{n+1} + \frac{M_1^n}{\Delta t} \psi^{n}
\end{array}
\end{array}
\right.
$$
where:
$$ [N_0^{k}]_{i,j} = \left(k^{-1}(\psi^{n+1}_k)\phi_j, \phi_i\right)_{L^2(\Omega)} \qquad [N_1^{k}]_{i,j} = \left(\frac{\partial k^{-1}(\psi^{n+1}_k)}{\partial \psi}\phi_j, \phi_i\right)_{L^2(\Omega)}$$
$$ [M_1^{k}]_{i,j} = \left(\frac{\partial \theta(\psi^{n+1}_k)}{\partial \psi} \gamma_j, \gamma_i\right)_{L^2(\Omega)} \qquad [M_2^{k}]_{i,j} = \left(\frac{\partial^2 \theta(\psi^{n+1}_k)}{\partial \psi^2} \gamma_j, \gamma_i\right)_{L^2(\Omega)} \qquad [M_1^{n}]_{i,j} = \left(\frac{\partial \theta(\psi^{n})}{\partial \psi} \gamma_j, \gamma_i\right)_{L^2(\Omega)}$$

In [15]:
div = pg.cell_mass(mdg) @ pg.div(mdg)

# get the degrees of freedom for each variable
dof_p, dof_q = div.shape
dof_psi = dof_p

In [16]:
def assemble_mass_psi_matrix(psi, der = dtheta_dpsi):
    return pg.mass_matrix(mdg, 0, P0) @ np.diag(der(psi))

In [17]:
def newton(sol_n, t_n_1):
    prev = sol_n.copy()
    sol_n_1 = prev

    # assemble the right-hand side
    fixed_rhs = np.zeros(dof_psi + dof_q)

    # Natural BCs
    fixed_rhs[:dof_q]    += np.hstack(list(cond(t_n_1) for cond in bc_value))

    # Gravity
    fixed_rhs[:dof_q]    += np.hstack(gravity)

    # Previous time step
    fixed_rhs[-dof_psi:] += assemble_mass_psi_matrix(prev[-dof_psi:]) @ prev[-dof_psi:] / dt


    for k in range(K):

        rhs = fixed_rhs.copy()

        pp.initialize_data(subdomain, data, key, {
            "second_order_tensor": conductivity_tensor(prev[-dof_psi:]),
        })

        # construct the local matrices
        n_0_k = pg.face_mass(mdg)
        m_1_k = assemble_mass_psi_matrix(prev[-dof_psi:])

        pp.initialize_data(subdomain, data, key, {
            "second_order_tensor": dconductivity_tensor_dpsi(prev[-dof_psi:]), # To fix...
        })
        n_1_k = pg.face_mass(mdg)
        m_2_k = assemble_mass_psi_matrix(prev[-dof_psi:], d2theta_dpsi)

        rhs[:dof_q]    -= n_1_k @ prev[:dof_q]
        rhs[-dof_psi:] -= m_2_k @ prev[-dof_psi:] / dt


        # assemble the saddle point problem
        spp = sps.bmat([[n_0_k, -div.T], 
                        [ div,  m_1_k / dt]], format="csc")

        # solve the problem
        ls = pg.LinearSystem(spp, rhs)
        ls.flag_ess_bc(np.hstack(bc_essential), np.zeros(dof_q + dof_psi))
    
        prev = sol_n_1.copy()
        sol_n_1 = ls.solve()

        err = np.sum(np.power(sol_n_1-prev, 2))

        print(f'Time {t_n_1}, Iteration {k}, error L2: {err}')

        if err < eps:
            return sol_n_1
    
    return sol_n_1

In [18]:
proj_q   = RT0.eval_at_cell_centers(subdomain)
proj_psi =  P0.eval_at_cell_centers(subdomain)

def export_step(save, current_sol, step) -> None:
    q =  current_sol[:dof_q]
    psi = current_sol[-dof_psi:]

    for _, data in mdg.subdomains(return_data=True):
        data[pp.STATE] = {"cell_q": (proj_q @ q).reshape((3, -1), order="F"), 
                          "cell_p": (proj_psi @ psi)}
    
    save.write_vtu(["cell_q", "cell_p"], time_step=step)

In [19]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [20]:
# assemble initial solution
initial_solution = np.zeros(dof_p + dof_q)
initial_solution[-dof_p:] += np.hstack(initial_pressure)

In [21]:
sol = [initial_solution]
save = pp.Exporter(mdg, "sol", folder_name=output_directory)
export_step(save, sol[-1], 0)

# Time Loop
for step in range(1, ceil(T/dt) + 1):
    sol.append( newton(sol[-1], step * dt) )
    export_step(save, sol[-1], step)

save.write_pvd(np.array(range(0, ceil(T/dt) + 1)) * dt)

/home/faccus/CLionProjects/porepy/src/porepy/numerics/vem/dual_elliptic.py:796: RuntimeWarning: invalid value encountered in true_divide
  return np.array([[K[1, 1], -K[0, 1]], [-K[0, 1], K[0, 0]]]) / det


[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.07113732 0.         0.        ]
 [0.         0.         0.         ... 0.         0.06985996 0.        ]
 [0.         0.         0.         ... 0.         0.         0.07113732]]
Time 0.020833333333333332, Iteration 0, error L2: nan


/home/faccus/.local/lib/python3.10/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:206: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.07113732 0.         0.        ]
 [0.         0.         0.         ... 0.         0.06985996 0.        ]
 [0.         0.         0.         ... 0.         0.         0.07113732]]
Time 0.020833333333333332, Iteration 1, error L2: nan
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Time 0.020833333333333332, Iteration 2, error L2: nan
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Time 0.020833333333333332, Iteration 3, error L2: nan
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [

KeyboardInterrupt: 